In [28]:
from ultralytics    \
    import YOLO

from math           \
    import (
        floor
)

from random         \
    import SystemRandom

from os.path        \
    import (
        join
)

In [29]:
ZERO: int = 0

In [30]:
EPOCHS: int = 100
BATCH: int  = 9
PATIENCE: int = 10

IMAGE_WIDTH_SIZE: int = 640
IMAGE_HEIGHT_SIZE: int = 640

IMAGE_DIMENSIONS: tuple = (
    IMAGE_HEIGHT_SIZE, 
    IMAGE_WIDTH_SIZE
)

SEED = None

IS_TO_RESUME: bool = False

MOSAIC_END_AT: int = -1

NUMBER_OF_TIMES_TO_ITERATE_TRAINING: int = 100

DEVICES_TO_USE: list = [
    ZERO
]

In [31]:
MODEL_WEIGHT_LOCATION: str | None = None
MODEL_ITERATION_VERSION: int = 3

In [32]:
def get_weight_location() -> str:
    global MODEL_WEIGHT_LOCATION
    return MODEL_WEIGHT_LOCATION

def set_weight_location(
    value:str
) -> None:
    global MODEL_WEIGHT_LOCATION
    MODEL_WEIGHT_LOCATION = value

def get_current_model_iteration() -> int:
    global MODEL_ITERATION_VERSION
    return MODEL_ITERATION_VERSION

def set_current_model_iteration(
    value: int
) -> None:
    global MODEL_ITERATION_VERSION
    MODEL_ITERATION_VERSION = value

def increment_of_model() -> None:
    set_current_model_iteration(
        get_current_model_iteration() + 1
    )

def generate_weight_location() -> str:
    weight_path = join(
        str(
            './runs/detect/train' 
            + 
            str(
                get_current_model_iteration()
            )
        ),
        'weights/best.pt'
    )

    set_weight_location(
        weight_path
    )

    return get_weight_location()

In [33]:
HOME_LOCATION: str = '/home/ai'
DATASET_LOCATION:   str = join(
    HOME_LOCATION,
    'dataset.yaml'
)

YOLO_MODEL_VERSION: str = 'yolo11m.pt'


MODEL_WEIGHT_LOCATION = generate_weight_location()

In [34]:
generator = None
model = None

In [35]:
model = YOLO(
    YOLO_MODEL_VERSION
)

if not (
    MODEL_WEIGHT_LOCATION is None
):
    model.load(
        get_weight_location()
    )

Transferred 643/649 items from pretrained weights


In [36]:
def getSeed() -> int:
    global SEED
    
    if isSeedNone():
        return generateSeed()

    return SEED

def  setSeed(
    value: int
) -> None:
    global SEED
    SEED = value


def generateGenerator() -> None:
    global generator

    if generator is None:
        generator = SystemRandom()

def isSeedNone() -> bool:
    global SEED
    return SEED is None

def generateSeed() -> None:
    global          \
        SEED,       \
        generator,  \
        ZERO
    
    generateGenerator()

    setSeed(
        generator.randint(
            ZERO, 
            2147483647
        )
    )

    return SEED

In [37]:
if not IS_TO_RESUME:
    generateSeed()

for idx in range(
    NUMBER_OF_TIMES_TO_ITERATE_TRAINING
):
    training_results =                          \
        model.train(
            data         = DATASET_LOCATION, 
            epochs       = EPOCHS,
            batch        = BATCH, 
            imgsz        = IMAGE_DIMENSIONS,
            device       = DEVICES_TO_USE,
            plots        = True,
            patience     = PATIENCE,
            close_mosaic = MOSAIC_END_AT,
            seed         = getSeed(),
            multi_scale  = False,
            resume       = IS_TO_RESUME,
            save         = True
    )
    
    if not (
        int( 
            NUMBER_OF_TIMES_TO_ITERATE_TRAINING 
            - 
            1 
        )
        == 
        idx
    ):
        increment_of_model()

        model.load(
            generate_weight_location()
        )


New https://pypi.org/project/ultralytics/8.3.71 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.70 🚀 Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/home/ai/dataset.yaml, epochs=10, time=None, patience=50, batch=9, imgsz=(640, 640), save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train3, exist_ok=False, pretrained=./runs/detect/train2/weights/best.pt, optimizer=auto, verbose=True, seed=15640295, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=-1, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None,

train: Scanning /tmp/dataset/training/labels.cache... 2205 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2205/2205 [00:00<?, ?it/s]
val: Scanning /tmp/dataset/validation/labels.cache... 945 images, 0 backgrounds, 0 corrupt: 100%|██████████| 945/945 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0004921875), 112 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train3
Starting training for 10 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      5.21G     0.2961     0.7874     0.9272         39        640: 100%|██████████| 245/245 [01:20<00:00,  3.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 53/53 [00:13<00:00,  3.94it/s]

                   all        945       1122      0.956      0.916      0.959      0.782


...

                   all        945       1122      0.977      0.946      0.974      0.851
                Cattle         22         22      0.981      0.909      0.957      0.845
                   Dog        942        942      0.995      0.993      0.995      0.961
              Humanoid        154        158      0.955      0.937      0.969      0.748
Speed: 0.2ms preprocess, 6.7ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train3
Transferred 649/649 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.71 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.70 🚀 Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/home/ai/dataset.yaml, epochs=10, time=None, patience=50, batch=9, imgsz=(640, 640), save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train32, exist_ok=False, pretrained=./runs/d

train: Scanning /tmp/dataset/training/labels.cache... 2205 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2205/2205 [00:00<?, ?it/s]
val: Scanning /tmp/dataset/validation/labels.cache... 945 images, 0 backgrounds, 0 corrupt: 100%|██████████| 945/945 [00:00<?, ?it/s]


Plotting labels to runs/detect/train32/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0004921875), 112 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train32
Starting training for 10 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      5.21G     0.3244     0.2452     0.9429         27        640:  10%|█         | 25/245 [00:08<01:17,  2.85it/s]


KeyboardInterrupt: 